In [69]:
from math import log, exp, sqrt
import numpy as np

In [70]:
## Prepare Data
from diabeticRetinopathyUtils import load_diabetic_retinopathy
X,y = load_diabetic_retinopathy(filename = "diabeticRetinopathy.csv")

## Q1.2

In [86]:
def f(w0,w,X,y):
    n = range(X.shape[0])
    rho = 1.0/X.shape[0]
    z = [X[i].dot(w)+w0 for i in n]
    
    #VALUE
    summands = [log(1+exp(-y[i]*z[i])) for i in n]
    func = sum(summands) + rho/2*np.linalg.norm(w)**2
        
    #GRADIENT
    summands = [-y[i]/(1+exp(y[i]*z[i])) for i in n]
    grad_w0 = sum(summands)
                        
    summands = [-X[i]*y[i]/(1+exp(y[i]*z[i])) for i in n]  
    grad_w = sum(summands) + rho*np.linalg.norm(w)
    
    #HESSIAN
    # Prepare a*a^T where a = [1,X[i]]
    a = [np.insert(X[i],0,1) for i in n]
    ex = [exp(y[i]*z[i]) for i in n]
    # Prepare a matrix with zeros in the first row and colomn, rho else
    sqrt_rho = sqrt(rho)*np.ones(19)
    sqrt_rho = np.insert(sqrt_rho,0,0)
    rho_matrix = np.outer(sqrt_rho,sqrt_rho)
    
    summands = [np.outer(a[i],a[i])*ex[i]*(y[i]/(1+ex[i])**2) for i in n]
    hessian = sum(summands) + rho_matrix
    
    return func,[grad_w0,grad_w],hessian
    
if False:
    w = np.random.randn(19)
    f(0,w,X,y)
    
    
#TODO check_gradient
    
    
    

## Q1.3

In [ ]:
def Newton(f,w0=0,w=0,X=X,y=y):
    